# Apply trace metal concentrations to river classification

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean

%matplotlib inline

## River categories:

1. Glaciers
2. Continental
3. Other

In [2]:
# Load classification file created in notebook "rivers---create_river_classification"
rclass = nc.Dataset('/ocean/brogalla/GEOTRACES/data/river_class-202005.nc','r')
river_class = rclass.variables['rclass']
river_class = np.array(river_class)

### Apply trace metal concentrations

Notebook to find these concentratons: rivers---find_characteristic_tracer_concentrations.ipynb

In [3]:
SPM_rivers = np.zeros(river_class.shape)
dMn_rivers = np.zeros(river_class.shape)

In [4]:
SPM_factor = 1

Manganese:

In [5]:
dMn_rivers[river_class==1.0] = 164e-9   # glaciers
dMn_rivers[river_class==2.0] = 29.8e-9  # continental
dMn_rivers[river_class==3.0] = 2.2e-9   # other
dMn_rivers[river_class==4.0] = 2.2e-9   # Gulf of Boothia

In [6]:
SPM_rivers[river_class==1.0] = SPM_factor*261e-6    # glaciers
SPM_rivers[river_class==2.0] = SPM_factor*1.196e-5  # continental
SPM_rivers[river_class==3.0] = SPM_factor*4.08e-6   # other
SPM_rivers[river_class==4.0] = SPM_factor*4.08e-6   # Gulf of Boothia

##### Write to NetCDF file

In [7]:
# Load structure from sample dataset of river runoff used in ANHA12
# from Paul Myers' group (http://knossos.eas.ualberta.ca/anha/anhatable.php)
c = nc.Dataset('/ocean/brogalla/GEOTRACES/data/runoff/ANHA12_runoff_monthly_combined_Dai_Trenberth_Bamber_y2002.nc','r')
lon_rf = np.array(c.variables['nav_lon'])
lat_rf = np.array(c.variables['nav_lat'])

In [14]:
ncd = nc.Dataset('/ocean/brogalla/GEOTRACES/data/paper1-forcing-files/river-forcing-base-20201125.nc', 'w', zlib=True)
ncd.createDimension('x',len(c.dimensions['x']))
ncd.createDimension('y',len(c.dimensions['y']))

# variables
nav_lat = ncd.createVariable('nav_lat', 'float32', ('y','x'))
nav_lat.long_name = 'Latitude'
nav_lat.units = 'degrees_north'
nav_lat[:] = lat_rf

nav_lon = ncd.createVariable('nav_lon', 'float32', ('y','x'))
nav_lon.long_name = 'Longitude'
nav_lon.units = 'degrees_east'
nav_lon[:] = lon_rf

spm_rivers = ncd.createVariable('spm_rivers', 'float32', ('y','x'))
spm_rivers.units = 'kg/L'
spm_rivers.long_name = 'Suspended Particulate Matter content'  
spm_rivers.coordinates = 'nav_lon nav_lat'
spm_rivers[:] = SPM_rivers

dmn_rivers = ncd.createVariable('dmn_rivers', 'float32', ('y','x'))
dmn_rivers.units = 'mol/L'
dmn_rivers.long_name = 'Dissolved Mn concentration'  
dmn_rivers.coordinates = 'nav_lon nav_lat'
dmn_rivers[:] = dMn_rivers

ncd.close()